In [1]:
%pip install --upgrade git+https://github.com/SheffieldML/PyDeepGP.git

  Cloning https://github.com/SheffieldML/PyDeepGP.git to /private/var/folders/nh/373y7nnj3q1d1bvtjs52wfpmp0cwgr/T/pip-req-build-h44_51wa
  Running command git clone -q https://github.com/SheffieldML/PyDeepGP.git /private/var/folders/nh/373y7nnj3q1d1bvtjs52wfpmp0cwgr/T/pip-req-build-h44_51wa
  Resolved https://github.com/SheffieldML/PyDeepGP.git to commit f2a1f568a7462633a58ed433520dcf7f0c98515c
     |################################| 987 kB 195 kB/s eta 0:00:01
  Created wheel for DGP: filename=DGP-1.0-py3-none-any.whl size=33248 sha256=732a0bc50d20570aa6908fc99c1292c7fe6e46807f771ed63a63729ab4cbe0a8
  Stored in directory: /private/var/folders/nh/373y7nnj3q1d1bvtjs52wfpmp0cwgr/T/pip-ephem-wheel-cache-5sr2662m/wheels/c0/61/cc/8ba13f739adcf108aa7de275f0bb8331e14f80230ae026c45f
Successfully built DGP
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install mlai

     |################################| 42 kB 515 kB/s eta 0:00:011
     |################################| 64 kB 1.7 MB/s eta 0:00:01
     |################################| 6.0 MB 309 kB/s eta 0:00:011     |##                              | 552 kB 36.0 MB/s eta 0:00:01
     |################################| 101 kB 9.8 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [18]:
%pip install gpy

Note: you may need to restart the kernel to use updated packages.


In [13]:
# Late bind setup methods to DeepGP object
from mlai.deepgp_tutorial import initialize
from mlai.deepgp_tutorial import staged_optimize
from mlai.deepgp_tutorial import posterior_sample
from mlai.deepgp_tutorial import visualize
from mlai.deepgp_tutorial import visualize_pinball
import mlai

import deepgp
deepgp.DeepGP.initialize=initialize
deepgp.DeepGP.staged_optimize=staged_optimize
deepgp.DeepGP.posterior_sample=posterior_sample
deepgp.DeepGP.visualize=visualize
deepgp.DeepGP.visualize_pinball=visualize_pinball

In [14]:
import numpy as np
import pandas as pd
import GPy
from tqdm import tqdm
from sklearn.model_selection import KFold

In [15]:
from sklearn.model_selection import train_test_split
import time
# X_train, X_test, y_train, y_test = train_test_split(df_loc, df_val, train_size=0.1, random_state=42)

In [16]:
def mse_func(y_pred,y_true):
    mse = np.mean((y_pred-y_true)**2)
    return mse

def mae_func(y_pred,y_true):
    mae = np.mean(np.absolute(y_pred-y_true))
    return mae

In [19]:
for data_no in range(5):
    df_loc = pd.read_csv("Sub-competition_2a/Training data/LOC_Train_"+str(data_no+1)+".csv",sep = ",", 
                         header = None, names=['x', 'y'])
    df_val = pd.read_csv("Sub-competition_2a/Training data/Z_Train_"+str(data_no+1)+".csv",sep = ",", 
                         header = None, names=['data'])
    df_loc,_,df_val,_ = train_test_split(df_loc, df_val,  
                                                test_size=0.7)
    print('------------------------------------------------------------------------')
    print(f'Training on dataset 2a_{data_no+1}...')
    N = len(df_loc)
    s = np.vstack((df_loc["x"],df_loc["y"])).T
    y = df_val.iloc[:,0].values
    ### Basis functions

    num_basis = [5**2,7**2,11**2]
    knots_1d = [np.linspace(0,1,int(np.sqrt(i))) for i in num_basis]
    ##Wendland kernel
    K = 0
    phi = np.zeros((N, sum(num_basis)))
    for res in range(len(num_basis)):
        theta = 1/np.sqrt(num_basis[res])*2.5
        knots_s1, knots_s2 = np.meshgrid(knots_1d[res],knots_1d[res])
        knots = np.column_stack((knots_s1.flatten(),knots_s2.flatten()))
        for i in range(num_basis[res]):
            d = np.linalg.norm(s-knots[i,:],axis=1)/theta
            for j in range(len(d)):
                if d[j] >= 0 and d[j] <= 1:
                    phi[j,i + K] = (1-d[j])**6 * (35 * d[j]**2 + 18 * d[j] + 3)/3
                else:
                    phi[j,i + K] = 0
        K = K + num_basis[res]
        ## Romove the all-zero columns
    idx_zero = np.array([], dtype=int)
    for i in range(phi.shape[1]):
        if sum(phi[:,i]!=0)==0:
            idx_zero = np.append(idx_zero,int(i))

    phi_reduce = np.delete(phi,idx_zero,1)
    
    num_folds = 3
    kfold = KFold(n_splits=num_folds, shuffle=True, random_state = 123)
    inputs = phi_reduce
    targets = y
    count = 0
    mse_vec = np.empty([num_folds,1])
    mae_vec = np.empty([num_folds,1])
    mpiw_vec = np.empty([num_folds,1])
    picp_vec = np.empty([num_folds,1])
    
    start_time = time.time()
    for train_idx, test_idx in kfold.split(inputs, targets):
        
        count += 1
        
        print(f'Cross-validation iteration {count} ...')
        
        hidden = 4
        m = deepgp.DeepGP([1,hidden,phi_reduce.shape[1]],Y=targets[train_idx], X=inputs[train_idx], 
                          inits=['PCA','PCA'], 
                          kernels=[GPy.kern.RBF(hidden,ARD=True),
                                   GPy.kern.RBF(phi.shape[1],ARD=True)], # the kernels for each layer
                          num_inducing=50, back_constraint=False)
        
        # Call the initalization
        m.initialize()
        for layer in m.layers:
            layer.likelihood.variance.constrain_positive(warning=False)
        m.optimize(messages=True,max_iters=1000)
        m.staged_optimize(messages=(True,True,True))
        y_test = targets[test_idx]
        pred, var = m.predict(inputs[test_idx])
        mse = mse_func(pred,y_test)  
        mae = mae_func(mean_vec1,y_test)
        # e = np.asarray(e)
        lower_bound1 = pred - 1.96*np.sqrt(var)
        upper_bound1 = pred + 1.96*np.sqrt(var)
        
        count_var0 = 0
        for i in range(len(y_test)):
            if ((y_test[i] > lower_bound1[i]) and (y_test[i] < upper_bound1[i])): count_var0 +=1
                
        picp= count_var0/len(targets[test_idx])
        mpiw = np.mean(upper_bound1 - lower_bound1)
        mse_vec[(count-1)] = mse
        mae_vec[(count-1)] = mae
        picp_vec[(count-1)] = picp
        mpiw_vec[(count-1)] = mpiw
        print(f'Performance on cross-validation {count}: MSE = {mse}, MAE = {mae}, picp = {picp}, mpiw = {mpiw}')
    
    end_time = time.time()
    mse = np.mean(mse_vec)
    mae = np.mean(mae_vec)
    picp = np.mean(picp_vec)
    mpiw = np.mean(mpiw_vec)
    time_t = (start_time - end_time)/num_folds
    print(f'Average performance on 2a-{data_no+1}: MSE = {mse}, MAE = {mae}, picp = {picp}, mpiw = {mpiw}, time = {-time_t}')

# Real data application

In [18]:
df = pd.read_csv("precip_data_real.csv",sep = ",")
df,_= train_test_split(df,test_size=0.3)
print('------------------------------------------------------------------------')
print(f'Training on precipitation dataset ...')
N = len(df)
s = np.vstack((df["x"],df["y"])).T
y = df.iloc[:,2].values
### Basis functions

num_basis = [5**2,7**2,11**2]
knots_1d = [np.linspace(0,1,int(np.sqrt(i))) for i in num_basis]
##Wendland kernel
K = 0
phi = np.zeros((N, sum(num_basis)))
for res in range(len(num_basis)):
    theta = 1/np.sqrt(num_basis[res])*2.5
    knots_s1, knots_s2 = np.meshgrid(knots_1d[res],knots_1d[res])
    knots = np.column_stack((knots_s1.flatten(),knots_s2.flatten()))
    for i in range(num_basis[res]):
        d = np.linalg.norm(s-knots[i,:],axis=1)/theta
        for j in range(len(d)):
            if d[j] >= 0 and d[j] <= 1:
                phi[j,i + K] = (1-d[j])**6 * (35 * d[j]**2 + 18 * d[j] + 3)/3
            else:
                phi[j,i + K] = 0
    K = K + num_basis[res]
    ## Romove the all-zero columns
idx_zero = np.array([], dtype=int)
for i in range(phi.shape[1]):
    if sum(phi[:,i]!=0)==0:
        idx_zero = np.append(idx_zero,int(i))

phi_reduce = np.delete(phi,idx_zero,1)

num_folds = 2
kfold = KFold(n_splits=num_folds, shuffle=True, random_state = 123)
inputs = phi_reduce
targets = y
count = 0
mse_vec = np.empty([num_folds,1])
mae_vec = np.empty([num_folds,1])
mpiw_vec = np.empty([num_folds,1])
picp_vec = np.empty([num_folds,1])

start_time = time.time()
for train_idx, test_idx in kfold.split(inputs, targets):

    count += 1

    print(f'Cross-validation iteration {count} ...')

    hidden = 4
    m = deepgp.DeepGP([1,hidden,phi_reduce.shape[1]],Y=targets[train_idx], X=inputs[train_idx], 
                      inits=['PCA','PCA'], 
                      kernels=[GPy.kern.RBF(hidden,ARD=True),
                               GPy.kern.RBF(phi.shape[1],ARD=True)], # the kernels for each layer
                      num_inducing=50, back_constraint=False)

    # Call the initalization
    m.initialize()
    for layer in m.layers:
        layer.likelihood.variance.constrain_positive(warning=False)
    m.optimize(messages=True,max_iters=1200)
#     m.staged_optimize(messages=(True,True,True))
    y_test = targets[test_idx]
    pred, var = m.predict(inputs[test_idx])
    mse = mse_func(pred,y_test)  
    mae = mae_func(mean_vec1,y_test)
    # e = np.asarray(e)
    lower_bound1 = pred - 1.96*np.sqrt(var)
    upper_bound1 = pred + 1.96*np.sqrt(var)

    count_var0 = 0
    for i in range(len(y_test)):
        if ((y_test[i] > lower_bound1[i]) and (y_test[i] < upper_bound1[i])): count_var0 +=1

    picp= count_var0/len(targets[test_idx])
    mpiw = np.mean(upper_bound1 - lower_bound1)
    mse_vec[(count-1)] = mse
    mae_vec[(count-1)] = mae
    picp_vec[(count-1)] = picp
    mpiw_vec[(count-1)] = mpiw
    print(f'Performance on cross-validation {count}: MSE = {mse}, MAE = {mae}, picp = {picp}, mpiw = {mpiw}')
    
end_time = time.time()
mse = np.mean(mse_vec)
mae = np.mean(mae_vec)
picp = np.mean(picp_vec)
mpiw = np.mean(mpiw_vec)
time_t = (start_time - end_time)/num_folds
print(f'Average performance on dataset: MSE = {mse}, MAE = {mae}, picp = {picp}, mpiw = {mpiw}, time = {-time_t}')